# Data preparation

In [14]:
import keras
import numpy as np
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


# Vectorizing sequences of characters

In [15]:
# Extracts the sequences of 60 charecters
maxlen = 60
# Sample new sequence based on number of steps
step = 3
# Holds our extracted sequences
sentences = []
# This holds the targets (the follow-up characters)
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)
# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


# Building the network

In [16]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

# Model Compilation

In [17]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Function to sample the next character given the model’s predictions

In [18]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Text-generation loop

In [19]:
import random
import sys
# trains the model for 20 epocs
for epoch in range(1, 20):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)
    
# Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
# Generates 400 characters starting from seed text in loop
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 196s 125ms/step - loss: 2.2704
--- Generating with seed: "essed of
color, skin and seeming. perhaps i could be fairly "
------ temperature: 0.2
essed of
color, skin and seeming. perhaps i could be fairly the same the sense of the sense of the sense of the sense of the sense of the sense of the soul, the sense of the sense of the sense of the present of the same the sense of the sense of the sense of the sense of the consert the sense of the some present of the sense of the sense of the sense of the sense of the sense of the sense of the seeps the present the sense the to the self-consentian in the
------ temperature: 0.5
eeps the present the sense the to the self-consentian in the restate the sentirivinity, from the consequent and intere histers that is seeps the old be the senter the condicine the the light to diffines to has the something with is been the soul, the fine to the disconsent of the feeling is the indivilual
profound to

<ipython-input-18-66c6eecb696d>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ducadimer
acxing to goder" in the circumscane! on vices of view saysupficing,
aration--who bestac od thousands to of con elesan
minds of "naic anevecthy.iness, however, when i
lisors
and wished goods
him, succcession ( vansction eponting with a cove? it seems everyone id: he decrarence of manyed controlacterans in humans
made or as eer suvest ak vyolity is gloce no
been u
epoch 19
1565/1565 [==============================] - 196s 125ms/step - loss: 1.3208
--- Generating with seed: "piritual men" of christianity have done for europe
hitherto!"
------ temperature: 0.2
piritual men" of christianity have done for europe
hitherto! it is the fact and the strengthers of the same thing in the same thing and man is not the present in the success of the superiorical problems of the successful and artistic of the superiorical sense and interests of the sense of the same thing that the such a standard of the sense of the fact that the such an enduring the such an explanation of the sense of the se